# Model serialization

## Import model

In [ ]:
from __future__ import annotations

In [ ]:
import json

with open("Lc2ppiK.json") as stream:
    model_def = json.load(stream)

In [ ]:
from IPython.display import JSON

JSON(model_def)

## Construct `ThreeBodyDecay`

In [ ]:
from typing import Callable
from warnings import warn

from ampform_dpd.decay import (
    FinalStateID,
    IsobarNode,
    Particle,
    State,
    ThreeBodyDecay,
    ThreeBodyDecayChain,
)


def to_decay(
    definition: dict, to_latex: Callable[[str], str] | None = None
) -> ThreeBodyDecay:
    distributions_def = model_def["distributions"]
    n_distributions = len(distributions_def)
    if n_distributions == 0:
        msg = "The serialized model does not have any distributions"
        raise ValueError(msg)
    if n_distributions > 1:
        msg = f"There are {n_distributions} distributions, but expecting one only"
        warn(msg, category=UserWarning)
    decay_description = distributions_def[0]["decay_description"]
    initial_state, final_state = _get_states(decay_description["kinematics"])
    return ThreeBodyDecay(
        states={
            initial_state.index: initial_state,
            **final_state,
        },
        chains=sorted({
            to_decay_chain(chain, initial_state, final_state, to_latex)
            for chain in decay_description["chains"]
        }),
    )


def to_decay_chain(
    chain_def: dict,
    initial_state: State,
    final_state: dict[FinalStateID, State],
    to_latex: Callable[[str], str] | None = None,
) -> ThreeBodyDecayChain:
    vertices = chain_def["vertices"]
    if to_latex is None:
        to_latex = lambda x: x  # noqa:E731
    resonance = Particle(
        name=chain_def["name"],
        latex=name_to_latex(chain_def["name"]),
        spin=chain_def["propagators"][0]["spin"],
        mass=0,
        width=0,
        parity=None,
    )
    return ThreeBodyDecayChain(
        decay=IsobarNode(
            parent=initial_state,
            child1=IsobarNode(
                parent=resonance,
                child1=final_state[vertices[1]["node"][0]],
                child2=final_state[vertices[1]["node"][1]],
            ),
            child2=final_state[vertices[0]["node"][1]],
        )
    )


def _get_states(kinematics: dict) -> tuple[State, dict[FinalStateID, State]]:
    initial_state_def = kinematics["initial_state"]
    final_state_def = kinematics["final_state"]
    initial_state = dict_to_particle(initial_state_def)
    final_state = {p["index"]: dict_to_particle(p) for p in final_state_def}
    return initial_state, final_state


def dict_to_particle(dct: dict) -> State:
    return State(
        name=dct["name"],
        latex=name_to_latex(dct["name"]),
        mass=dct["mass"],
        width=0,
        spin=dct["spin"],
        parity=None,
        index=dct["index"],
    )

In [ ]:
def name_to_latex(name: str) -> str:
    latex = {
        "Lc": R"\Lambda_c^+",
        "pi": R"\pi^+",
        "K": "K^-",
        "p": "p",
    }.get(name)
    if latex is not None:
        return latex
    mass_str = name[1:].strip("(").strip(")")
    subsystem_letter = name[0]
    subsystem = {"D": "D", "K": "K", "L": R"\Lambda"}.get(subsystem_letter)
    if subsystem is None:
        return name
    return f"{subsystem}({mass_str})"


decay = to_decay(model_def, to_latex=name_to_latex)

In [ ]:
from IPython.display import Math

from ampform_dpd.io import aslatex

Math(aslatex(decay))

In [ ]:
from IPython.display import Markdown

from ampform_dpd.io import as_markdown_table

Markdown(as_markdown_table(decay))